# Importing the relevant libraries

In [3]:
import os
import backtrader as bt
import pandas as pd
import numpy as np
from datetime import datetime

# Load Stock Data from CSV and Combine Datasets from Multiple Folders
This function reads a CSV file containing stock market data, converts the timestamp column to a proper datetime format, and ensures that the required columns ('open', 'high', 'low', 'close', and 'volume') are present. It raises an error if these columns are missing. Then navigates through multiple directories containing stock CSV files, loading and concatenating them into a single DataFrame for further processing. It skips files missing critical columns.



In [6]:

def load_stock_data(file_path):
    """Load stock data from CSV and ensure proper time formatting."""
    df = pd.read_csv(file_path)
    if 'timestamp' in df.columns:
        if df['timestamp'].apply(lambda x: str(x).isdigit()).all():
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        else:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
    elif 'date' in df.columns:
        df['timestamp'] = pd.to_datetime(df['date'])
    else:
        raise KeyError(f"No valid time column found in file: {file_path}")

    df.set_index('timestamp', inplace=True)

    required_columns = ['open', 'high', 'low', 'close', 'volume']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise KeyError(f"Missing required columns {missing_columns} in file: {file_path}")

    return df

def combine_datasets(data_folders):
    """Combine stock data from multiple folders into one DataFrame."""
    combined_data = pd.DataFrame()
    for folder in data_folders:
        for subdir, _, files in os.walk(folder):
            for file in files:
                if file.endswith('.csv'):
                    file_path = os.path.join(subdir, file)
                    print(f'Loading {file_path}')
                    try:
                        df = load_stock_data(file_path)
                        combined_data = pd.concat([combined_data, df])
                    except KeyError as e:
                        print(f"Skipping {file_path}: {e}")
                        continue
    return combined_data

# Alpha3Strategy Class - Strategy Implementation
This is a custom trading strategy implemented using Backtrader. The strategy ranks 'open' prices and 'volume' over a given lookback period, calculates the correlation between these ranks, and generates buy/sell signals based on the alpha signal.


In [8]:
class Alpha3Strategy(bt.Strategy):
    params = (
        ('lookback_period', 10),  # Lookback period for correlation
    )

    def __init__(self):
        self.alpha_signal = dict()
        self.trade_pnls = []  # Track P&L for each trade

    def next(self):
        # Get the last `lookback_period` values for open and volume
        opens = np.array([self.data.open[-i] for i in range(self.params.lookback_period)])
        volumes = np.array([self.data.volume[-i] for i in range(self.params.lookback_period)])

        # Rank the values
        open_rank = pd.Series(opens).rank()
        volume_rank = pd.Series(volumes).rank()

        # Calculate correlation between ranked open and ranked volume
        correlation = open_rank.corr(volume_rank)

        # The alpha signal is the negative correlation
        alpha_signal = -1 * correlation

        # Implement simple buy/sell rules based on alpha signal
        if alpha_signal < -0.5:  # Strong negative correlation, buy signal
            if not self.position:
                self.buy()
        
        if alpha_signal > 0.5:  # Weak signal or positive correlation, sell signal
            if self.position:
                self.sell()

    def notify_trade(self, trade):
        """Capture the trade P&L when a trade closes."""
        if trade.isclosed:
            self.trade_pnls.append(trade.pnl)  # Store trade P&L
            


# Run Backtest on Portfolio Data
This function runs the backtest by adding stock data to the Backtrader engine. It simulates trading using the developed Alpha3 strategy and includes transaction costs and slippage to make the results more realistic.


In [10]:
def run_alpha3_backtest_portfolio(data_folders):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(Alpha3Strategy)

    for folder in data_folders:
        for subdir, _, files in os.walk(folder):
            for file in files:
                if file.endswith('1h.csv'):
                    file_path = os.path.join(subdir, file)
                    print(f'Loading {file_path}')
                    try:
                        df = load_stock_data(file_path)
                    except KeyError as e:
                        print(f"Skipping {file_path}: {e}")
                        continue

                    data_feed = bt.feeds.PandasData(dataname=df)
                    cerebro.adddata(data_feed)

    cerebro.broker.set_cash(1000000)
    cerebro.broker.setcommission(commission=0.001)
    cerebro.broker.set_slippage_perc(0.001)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Generate performance report
    generate_performance_report(cerebro)

    # Plot the backtest results
    cerebro.plot(iplot=False)

# Generate Performance Report
After running the backtest, this function generates key performance metrics, including total return, Sharpe ratio, Sortino ratio, maximum drawdown, trade frequency, and hit ratio. This is essential for understanding the strategy's effectiveness.


In [18]:
def generate_performance_report(cerebro):
    """Generate detailed performance metrics after backtesting."""
    portfolio_value = cerebro.broker.getvalue()
    initial_value = 1000000

    returns = (portfolio_value - initial_value) / initial_value * 100
    print('Total Portfolio Value: %.2f' % portfolio_value)
    print('Total Returns: %.2f%%' % returns)

    # Get the trade P&L from the first strategy in the first list
    strategy = cerebro.runstrats[0][0]  # Correctly access the first strategy
    returns_series = np.array(strategy.trade_pnls)
    daily_returns = returns_series / initial_value

    sharpe_ratio = np.mean(daily_returns) / np.std(daily_returns) if np.std(daily_returns) != 0 else 0
    sortino_ratio = np.mean(daily_returns[daily_returns > 0]) / np.std(daily_returns[daily_returns > 0]) if np.std(daily_returns[daily_returns > 0]) != 0 else 0
    
    print('Sharpe Ratio: %.2f' % sharpe_ratio)
    print('Sortino Ratio: %.2f' % sortino_ratio)

    max_drawdown = max(1 - (min(daily_returns) / max(daily_returns)), 0)
    print('Maximum Drawdown: %.2f%%' % (max_drawdown * 100))

    trade_count = len(strategy.trade_pnls)
    print('Trade Frequency: %d trades' % trade_count)

    winning_trades = len([pnl for pnl in strategy.trade_pnls if pnl > 0])
    hit_ratio = winning_trades / trade_count if trade_count > 0 else 0
    print('Hit Ratio: %.2f%%' % (hit_ratio * 100))

In [ ]:
if __name__ == '__main__':
    data_folders = [
        r'C:\Users\adity\OneDrive\Desktop\Quant trading project\task_dataset\datasets\quant\portfolio-universe\Indian'
    ]
    
    run_alpha3_backtest_portfolio(data_folders)

Loading C:\Users\adity\OneDrive\Desktop\Quant trading project\task_dataset\datasets\quant\portfolio-universe\Indian\GODREJIND\GODREJIND_1h.csv
Loading C:\Users\adity\OneDrive\Desktop\Quant trading project\task_dataset\datasets\quant\portfolio-universe\Indian\NIFTY 50\NIFTY 50_1h.csv
Loading C:\Users\adity\OneDrive\Desktop\Quant trading project\task_dataset\datasets\quant\portfolio-universe\Indian\NIFTY BANK\NIFTY BANK_1h.csv
Loading C:\Users\adity\OneDrive\Desktop\Quant trading project\task_dataset\datasets\quant\portfolio-universe\Indian\RELIANCE\RELIANCE_1h.csv
Loading C:\Users\adity\OneDrive\Desktop\Quant trading project\task_dataset\datasets\quant\portfolio-universe\Indian\TATAMOTORS\TATAMOTORS_1h.csv
Loading C:\Users\adity\OneDrive\Desktop\Quant trading project\task_dataset\datasets\quant\portfolio-universe\Indian\TCS\TCS_1h.csv
Starting Portfolio Value: 1000000.00
Ending Portfolio Value: 999891.45
Total Portfolio Value: 999891.45
Total Returns: -0.01%
Sharpe Ratio: 0.02
Sortino R